### Load Environment & Imports

In [1]:
# Load .env and import libs
from dotenv import load_dotenv
import os
from pydantic import BaseModel, UUID4, Field
import uuid
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessageChunk


In [2]:
# Load API key from .env file into environment
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
print("API key loaded:", bool(api_key))  # Test: Should print True if key loaded

API key loaded: True


### Define Pydantic Message Schema

In [4]:
class ChatMessage(BaseModel):
    id: UUID4 = Field(default_factory=uuid.uuid4)
    sender: str  # 'human' or 'ai'
    message: str

In [7]:
# Test creating a valid message
msg = ChatMessage(sender="human", message="Hello!")
print(msg)

id=UUID('630fd821-f94a-4556-952d-316c4732496e') sender='human' message='Hello!'


### Initialize LangChain Chat Model using init_chat_model

In [15]:
chat_model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=api_key,
    temperature=0.3,
)

In [9]:
# Test: Check if chat_model supports invoke()
print("Has invoke method?", hasattr(chat_model, "invoke"))

Has invoke method? True


### Convert Pydantic Messages to LangChain HumanMessage

In [10]:
def convert_to_lc_messages(messages: list[ChatMessage]):
    return [HumanMessage(content=msg.message) for msg in messages if msg.sender == "human"]


In [11]:
# Test:
input_msgs = [
    ChatMessage(sender="human", message="Hello there!"),
    ChatMessage(sender="ai", message="Hi! How can I help?")
]

lc_msgs = convert_to_lc_messages(input_msgs)
print(lc_msgs)  # Should print list with only human messages as HumanMessage objects

[HumanMessage(content='Hello there!', additional_kwargs={}, response_metadata={})]


### Chat function — send messages and get AI reply

In [16]:
def chat_with_model(messages: list[ChatMessage], chat_model):
    lc_messages = convert_to_lc_messages(messages)
    response = chat_model.invoke(lc_messages)
    return ChatMessage(sender="ai", message=response.content)

# Test chat function
history = [
    ChatMessage(sender="human", message="What is LangChain?"),
    ChatMessage(sender="human", message="Explain simply.")
]

ai_resp = chat_with_model(history, chat_model)
print("[AI]", ai_resp.message)


[AI] LangChain is a framework designed to help developers build applications that use language models, like those from OpenAI. It provides tools and components to make it easier to integrate these models into various applications, allowing developers to create chatbots, question-answering systems, and other language-based functionalities. Essentially, LangChain simplifies the process of working with language models by offering pre-built modules and workflows.
